In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import pandas as pd
from datetime import datetime
import numpy as np
from requests_html import HTMLSession
import psycopg2 as pg2

In [2]:
main_link = 'https://www.kariera.gr/en/jobs?title=&page=0&limit=20'

In [3]:
main_link = 'https://www.kariera.gr/en/jobs?title=&page=0&limit=20'
headers = {'User-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0'}
r = requests.get(main_link, headers=headers)
r

<Response [200]>

## Main Idea
### scrape all main_link + sub_links where sub_links=data%20analyst,data%20scientist,data%20engineer

In [4]:
soup = BeautifulSoup(r.content, 'html.parser')

In [5]:
max_page = soup.find('li',class_='ant-pagination-next').previous_sibling.text
max_pages = int(max_page)

In [6]:
max_pages

155

### working out how to turn pages:

In [7]:
main_link = 'https://www.kariera.gr/en/jobs?title=&page=0&limit=20'

In [8]:
main_link.split('&')[-2][-1]

'0'

In [9]:
page_link_list=[]
for i in range(0,max_pages+1):
    page_link_list.append('https://www.kariera.gr/en/jobs?title=&page={}&limit=20'.format(i))

## getting all sub-links from all links

In [10]:
link_list=[]
session = HTMLSession()
for i in tqdm(page_link_list):
    for y in session.get(i).html.links:
        if '/en/jobs/' in y:
            link_list.append(y)

  0%|          | 0/156 [00:00<?, ?it/s]

In [11]:
len(link_list)

3089

In [12]:
final_list=[]
main = 'https://www.kariera.gr/'
for i in link_list:
    final_list.append(main + i)

In [17]:
test_link = final_list[1]

In [18]:
headers = {'User-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0'}
r = requests.get(test_link, headers=headers)
soup = BeautifulSoup(r.content, 'html.parser')
soup

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="initial-scale=1.0, width=device-width" name="viewport"/><meta content="website" property="og:type"/><meta content="0kddlwbsdagaz58pjd2fj7p4wuuroo" name="facebook-domain-verification"/><link as="image" imagesrcset="/_next/image?url=%2Fassets%2Fimages%2Fflag-en.png&amp;w=32&amp;q=75 1x, /_next/image?url=%2Fassets%2Fimages%2Fflag-en.png&amp;w=48&amp;q=75 2x" rel="preload"/><title>Real Estate University: Εργάζεσαι &amp; Εκπαιδεύεσαι ταυτόχρονα στη Νο1 Εταιρεία Real Estate Παγκοσμίως (Βόρεια, Δυτικά Προάστια, Πειραιάς ή Προάστια Πειραιά) - 41833</title><script type="application/ld+json">{"@context":"http://schema.org","@type":"JobPosting","datePosted":"2023-01-08T04:24:28.000Z","description":"<p>Καριέρα στο real estate; Θέλεις να εργάζεσαι με ελεύθερο ωράριο, πολλαπλές δυνατότητες εξέλιξης και αμοιβές που αγγίζουν το 100% (!) της προμήθειας; Έλα τότε στη No1 Εταιρεία Real Estate παγκοσμίως!</p>\n<h3><strong data-str

### job title

In [19]:
soup.find('div',class_='h4 UGhAL2CE').text

'Real Estate University: Εργάζεσαι & Εκπαιδεύεσαι ταυτόχρονα στη Νο1 Εταιρεία Real Estate Παγκοσμίως (Βόρεια, Δυτικά Προάστια, Πειραιάς ή Προάστια Πειραιά)'

### company

In [20]:
soup.find('a',class_='__8_RJM1pE').text

'KELLER WILLIAMS SOLUTIONS GROUP'

### location

In [21]:
soup.findAll('a',class_='V_oVSppD main-body-text')[0].text

'Marousi'

### job occupation

In [22]:
getattr(soup.findAll('a',class_='V_oVSppD main-body-text')[3],'text',None)

'Real Estate'

In [23]:
soup.findAll('a',class_='V_oVSppD main-body-text')[3].text

'Real Estate'

### level

In [24]:
soup.findAll('a',class_='V_oVSppD main-body-text')[1].text

'Mid level'

### job type

In [25]:
soup.findAll('a',class_='V_oVSppD main-body-text')[2].text

'Contract/Freelance/Self-employed'

### content

In [26]:
result = soup.findAll('div',class_='__2DY4wJ3z hi8OBmAZ')
a=''
for res in result:
    a= (res.get_text(strip=True,separator=' '))
    a = a.replace('\u202f','').replace('\xa0','').replace('/','')
a

'Καριέρα στο real estate; Θέλεις να εργάζεσαι με ελεύθερο ωράριο, πολλαπλές δυνατότητες εξέλιξης και αμοιβές που αγγίζουν το 100% (!) της προμήθειας; Έλα τότε στη No1 Εταιρεία Real Estate παγκοσμίως! Ποιοι είμαστε: Η Keller Williams Solutions Group * έχει καταφέρει μέσα σε 6 χρόνια λειτουργίας να αριθμεί 25 Συμβούλους και να εντάσσεται στις πρώτες θέσεις πανελλαδικά σε πωλήσεις. Βασικός πυλώνας της επιτυχίας μας είναι η φροντίδα και προσοχή που δίνουμε στην ανάπτυξη των συνεργατώνΣυμβούλων μας. Τα αποτελέσματά μας μιλούν από μόνα τους: 1 Πώληση  4 ημέρες για τα έτη 2017-2022 Οι Σύμβουλοί μας κατατάσσονται στους κορυφαίους πανελλαδικά μέσα σε 2-3 χρόνια στο χώρο. Για το 2021: Οι 2 πρώτοι Σύμβουλοι πανελλαδικά ανήκουν στο γραφείο μας Αποστολή μας είναι να «Ενδυναμώνουμε τους ανθρώπους να παίρνουν συνειδητοποιημένες αποφάσεις σχετικά με την ακίνητη περιουσία τους», υιοθετώντας ταυτόχρονα μια σύγχρονη προσέγγιση στον κλάδο του real estate και στοχεύοντας να γίνουμε το Νο.1 Γραφείο πανελλαδ

# building the loop

In [27]:
df2 = pd.DataFrame()

In [28]:
for i in tqdm(final_list):
    link = i
    unique_id = i[-5:]
    a=''
    r = requests.get(link, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    if soup.find('div',class_='h4 UGhAL2CE') is not None:
        x = soup.find('div',class_='h4 UGhAL2CE').text
        if ('Data Analyst' in x) | ('Data Scientist' in x) | ('Data Engineer' in x): 
            try:
                job_title = soup.find('div',class_='h4 UGhAL2CE').text
            except:
                job_title = np.nan
            try:
                company = soup.find('a',class_='__8_RJM1pE').text
            except:
                company = np.nan
            try:
                location = soup.findAll('a',class_='V_oVSppD main-body-text')[0].text
            except:
                location = np.nan
            try:
                job_occupation = soup.findAll('a',class_='V_oVSppD main-body-text')[3].text
            except:
                job_occupation = np.nan
            try:
                level = soup.findAll('a',class_='V_oVSppD main-body-text')[1].text
            except:
                level = np.nan
            try:
                job_type = soup.findAll('a',class_='V_oVSppD main-body-text')[2].text
            except:
                job_type = np.nan
            try:
                result = soup.findAll('div',class_='__2DY4wJ3z hi8OBmAZ')
                for i in result:
                    a= i.get_text(strip=True,separator=' ')
                    a = a.replace('\u202f','').replace('\xa0','').replace('/','')
            except:
                content = np.nan
            temp = pd.DataFrame([{
                    'job_id' : unique_id,
                    'job_title' : job_title,
                    'company' : company,
                    'location': location,
                    'job_occupation' : job_occupation,
                    'level' : level,
                    'job_type' : job_type,
                    'content' : a,
                    'date_scraped' : datetime.now().strftime("%Y-%m-%d %H:%M")
                }])
            df2 = pd.concat([df2, temp], ignore_index=True)

  0%|          | 0/3089 [00:00<?, ?it/s]

In [29]:
df2

,job_id,job_title,company,location,job_occupation,level,job_type,content,date_scraped
0,41404,Data Engineer,DEMO AEBE\t,Krioneri,IT,Mid level,Full-time,"Description At Demo SA, one of the world’s lar...",2023-01-08 13:31
1,40408,Data Engineer - Spark Developer,EUROPEAN DYNAMICS,Marousi,IT,Mid level,Full-time,We currently have a vacancy position for a Dat...,2023-01-08 13:38
2,40269,Senior Data Engineer,Κωτσόβολος - Dixons South East Europe,Athina,IT,Senior,Full-time/Part-time,"ΠΕΡΙΓΡΑΦΗ ΘΕΣΗΣ ΕΡΓΑΣΙΑΣ In Kotsovolos, a memb...",2023-01-08 13:39
3,40268,Data Engineer,Κωτσόβολος - Dixons South East Europe,Athina,IT,Mid level,Full-time/Part-time,"ΠΕΡΙΓΡΑΦΗ ΘΕΣΗΣ ΕΡΓΑΣΙΑΣ In Kotsovolos, a memb...",2023-01-08 13:39
4,40260,Senior Mobile Data Engineer,Volton Ελληνική Ενεργειακή Α.Ε.,Metamorfosi,Engineering,Senior,Full-time,Volton Hellenic Energy SA is a Greek company t...,2023-01-08 13:39
5,40270,Senior Data Scientist,Κωτσόβολος - Dixons South East Europe,Athina,IT,Senior,Full-time/Part-time,"ΠΕΡΙΓΡΑΦΗ ΘΕΣΗΣ ΕΡΓΑΣΙΑΣ In Kotsovolos, a memb...",2023-01-08 13:39
6,39517,Senior BI & Data Engineer,WATT AND VOLT A.E.,Marousi,IT,Senior,Full-time,Description WATT+VOLT is one of the largest en...,2023-01-08 13:45
7,38344,Data Analyst,Mellon Technologies,Pireas,IT,Mid level,Full-time,Description Responsible for the quality and co...,2023-01-08 13:53


In [33]:
name1 = 'content' + '_' + str(datetime.now().strftime("%d-%m-%Y") + '.csv')

In [34]:
df2.to_csv(name1, index=False)

## Connect to postgres

In [35]:
conn = pg2.connect(database='Jobs',user='postgres',password='new_password')

In [36]:
cur = conn.cursor()

In [37]:
def create_table(cur):
    create_table_command = (""" 
    CREATE TABLE IF NOT EXISTS jobs (
    job_id int primary key NOT NULL,
   job_title text NOT NULL,
   company text,
   location text,
   job_occupation text,
   level text,
   job_type text,
   content text,
   date_scraped timestamp NOT NULL
    );
    """)
    
    cur.execute(create_table_command)

In [38]:
create_table(cur)

In [39]:
def check_if_job_exists(cur,job_id):
    query = (""" select job_id from jobs where job_id = %s """)
    cur.execute(query,(job_id,))
    
    return cur.fetchone() is not None

In [40]:
def update_db(cur, df):

    l = []
    for row in df.iterrows():
        if check_if_job_exists(cur, row[1][0]):
            pass
        else:
            l.append(row[1][0])

    return l

# Create dataframe with new jobs

In [41]:
new_jobs = update_db(cur,df2)

In [42]:
new_jobs = df2[df2['job_id'].apply(lambda x : x in new_jobs)]

In [43]:
new_jobs

,job_id,job_title,company,location,job_occupation,level,job_type,content,date_scraped


## Push new jobs to postgress

In [44]:
def insert_into_table(cur,job_id, job_title, company, location, job_occupation, level,job_type, content, date_scraped):
    insert_new_jobs = (
        """ INSERT INTO jobs (job_id, job_title, company, location, job_occupation, level,
           job_type, content, date_scraped)
            VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s);""")

    job_to_insert = (job_id, job_title, company, location, job_occupation, level,
                     job_type, content, date_scraped)

    cur.execute(insert_new_jobs,job_to_insert)

In [45]:
def push_from_df_to_db(cur, df):
    for i, row in df.iterrows():
        insert_into_table(cur, row['job_id'], row['job_title'], row['company'],
                          row['location'], row['job_occupation'], row['level'],
                          row['job_type'], row['content'], row['date_scraped'])

In [46]:
push_from_df_to_db(cur, new_jobs)

In [47]:
conn.commit()

In [48]:
conn.close()

# Fetch jobs table and save to a df

In [330]:
conn = pg2.connect(database='Jobs', user='postgres', password='new_password')

with conn:

    cur = conn.cursor()
    cur.execute("SELECT * FROM jobs")

    rows = cur.fetchall()

df = pd.DataFrame(rows).rename(
    columns={
        0: 'job_id',
        1: 'job_title',
        2: 'company',
        3: 'location',
        4: 'job_occupation',
        5: 'level',
        6: 'job_type',
        7: 'content',
        8: 'date_scraped'
    })

conn.close()

In [331]:
df.head()

,job_id,job_title,company,location,job_occupation,level,job_type,content,date_scraped
0,38344,Data Analyst,Mellon Technologies,Pireas,IT,Mid level,Full-time,Description Responsible for the quality and co...,2022-12-17 01:39:00
1,37348,Data Engineer - Patras,Ernst and Young,Patras,IT,Mid level,Full-time,"Requisition ID: 146677 At EY, you’ll have the ...",2022-12-17 01:47:00
2,37357,Junior Data Engineer,Ernst and Young,Athina,IT,Mid level,Full-time,Requisition ID: 127061 Junior Data Engineer At...,2022-12-17 01:47:00
3,37349,Data & Analytics Manager - Data Engineering,Ernst and Young,Athina,IT,Senior,Full-time,"Requisition ID: 814902 At EY, you’ll have the ...",2022-12-17 01:47:00
4,37345,Data Scientist - Patras,Ernst and Young,Patras,IT,Mid level,Full-time,"Requisition ID: 147180 At EY, you’ll have the ...",2022-12-17 01:47:00


## Cleaning company names

In [332]:
df['company'] = df['company'].apply(lambda x : x.replace('\t','').strip())

## Categorizing jobs

In [333]:
df['categories'] = df['job_title'].apply(lambda x: 'Analytics' if 'Analyst' in x else
                      ('Engineering' if 'Engineer' in x else 'Science'))

## Cleaning job_title

In [334]:
# removing spaces
df['job_title'] = df['job_title'].apply(lambda x : x.strip())

In [335]:
# removing locations from job titles
# it assumes that always all locations will be preceded by ' - '
def clean_city_from_job(job,city):
    titles = []
    for i,j in zip(job,city):
        if j in i:
            title = i[:-(3+len(j))]
            titles.append(title)
        else:
            title = i
            titles.append(title)
    return pd.DataFrame(titles).rename(columns={0:'job_title'})

In [336]:
df = clean_city_from_job(df['job_title'], df['location']).merge(
    df, left_index=True,
    right_index=True).drop('job_title_y',
                           axis=1).rename(columns={'job_title_x': 'job_title'})

In [337]:
df.head()

,job_title,job_id,company,location,job_occupation,level,job_type,content,date_scraped,categories
0,Data Analyst,38344,Mellon Technologies,Pireas,IT,Mid level,Full-time,Description Responsible for the quality and co...,2022-12-17 01:39:00,Analytics
1,Data Engineer,37348,Ernst and Young,Patras,IT,Mid level,Full-time,"Requisition ID: 146677 At EY, you’ll have the ...",2022-12-17 01:47:00,Engineering
2,Junior Data Engineer,37357,Ernst and Young,Athina,IT,Mid level,Full-time,Requisition ID: 127061 Junior Data Engineer At...,2022-12-17 01:47:00,Engineering
3,Data & Analytics Manager - Data Engineering,37349,Ernst and Young,Athina,IT,Senior,Full-time,"Requisition ID: 814902 At EY, you’ll have the ...",2022-12-17 01:47:00,Engineering
4,Data Scientist,37345,Ernst and Young,Patras,IT,Mid level,Full-time,"Requisition ID: 147180 At EY, you’ll have the ...",2022-12-17 01:47:00,Science


## Cleaning content

In [338]:
df['content'] = df['content'].apply(
    lambda x: x.replace('Description', '').replace('Company', '').replace(
        'ΠΕΡΙΓΡΑΦΗ ΘΕΣΗΣ ΕΡΓΑΣΙΑΣ', '').replace('Job', '').replace(
            '’s', '').replace('Purpose', '').strip())

In [339]:
import re

In [340]:
df['content'] = df['content'].apply(lambda x : re.sub(r'Requisition ID: \d+', '', x).strip())

## Geolocate locations

In [341]:
from geopy import geocoders
import swifter
gn = geocoders.GeoNames('asterios')

In [342]:
df['geo'] = df['location'].swifter.apply(lambda x : gn.geocode(x)).apply(lambda loc: tuple(loc.point) if loc else None)

Pandas Apply:   0%|          | 0/27 [00:00<?, ?it/s]

In [343]:
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['geo'].tolist(), index=df.index)

In [344]:
df.head(2)

,job_title,job_id,company,location,job_occupation,level,job_type,content,date_scraped,categories,geo,latitude,longitude,altitude
0,Data Analyst,38344,Mellon Technologies,Pireas,IT,Mid level,Full-time,Responsible for the quality and completeness o...,2022-12-17 01:39:00,Analytics,"(37.94203, 23.64619, 0.0)",37.94203,23.64619,0.0
1,Data Engineer,37348,Ernst and Young,Patras,IT,Mid level,Full-time,"At EY, you’ll have the chance to build a caree...",2022-12-17 01:47:00,Engineering,"(38.24444, 21.73444, 0.0)",38.24444,21.73444,0.0


### At this point we are done with the cleaning next up is droping unwanted columns and exporting  the csv for the main app

In [346]:
df.drop(['job_id','job_occupation','date_scraped','geo','altitude'],axis=1,inplace=True)

In [348]:
df.to_csv('job_board.csv',index=False)